In [34]:
#pip install streamlit

In [35]:
#pip install plotly


In [36]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.graph_objects as go
from plotly import tools
import plotly.offline as py
import plotly.express as px

In [37]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [38]:
vaccinations = spark.read.load("./data/Vaccinations/", format="csv", header=True, inferSchema="True")
vaccinations.printSchema()

root
 |-- dep: string (nullable = true)
 |-- clage_vacsi: integer (nullable = true)
 |-- jour: string (nullable = true)
 |-- n_dose1: integer (nullable = true)
 |-- n_cum_dose1: integer (nullable = true)



vaccinations.groupBy(["dep", "clage_vacsi"]).sum("n_dose1").sort(["dep", "clage_vacsi"]).take(100)

# Nombre de vaccinées par tranche d'âge

In [39]:
vacc = spark.read.load("./data/Vaccinations/", format="csv", header=True, inferSchema="True")
#vacc.printSchema()

In [40]:
newvacc = vacc.select(["clage_vacsi", "n_cum_dose1"])
#newvacc.collect()

In [41]:
nbVaccByAge = newvacc.where( newvacc.clage_vacsi != "0").groupBy("clage_vacsi").agg({"n_cum_dose1" : "max"}).sort(["clage_vacsi"])

In [42]:
#newvacc.where( newvacc.clage_vacsi == "0").groupBy("clage_vacsi").agg({"n_cum_dose1" : "max"}).sort(["clage_vacsi"]).collect()

In [43]:
nbVaccByAge.show()


+-----------+----------------+
|clage_vacsi|max(n_cum_dose1)|
+-----------+----------------+
|         24|             325|
|         29|             809|
|         39|            2218|
|         49|            3169|
|         59|            9925|
|         64|            4356|
|         69|            3364|
|         74|            3636|
|         79|            9318|
|         80|           17667|
+-----------+----------------+



In [44]:
nbVaccByAge.collect()


[Row(clage_vacsi=24, max(n_cum_dose1)=325),
 Row(clage_vacsi=29, max(n_cum_dose1)=809),
 Row(clage_vacsi=39, max(n_cum_dose1)=2218),
 Row(clage_vacsi=49, max(n_cum_dose1)=3169),
 Row(clage_vacsi=59, max(n_cum_dose1)=9925),
 Row(clage_vacsi=64, max(n_cum_dose1)=4356),
 Row(clage_vacsi=69, max(n_cum_dose1)=3364),
 Row(clage_vacsi=74, max(n_cum_dose1)=3636),
 Row(clage_vacsi=79, max(n_cum_dose1)=9318),
 Row(clage_vacsi=80, max(n_cum_dose1)=17667)]

In [45]:
type(nbVaccByAge)

pyspark.sql.dataframe.DataFrame

In [46]:
nbVaccByAge.show()


+-----------+----------------+
|clage_vacsi|max(n_cum_dose1)|
+-----------+----------------+
|         24|             325|
|         29|             809|
|         39|            2218|
|         49|            3169|
|         59|            9925|
|         64|            4356|
|         69|            3364|
|         74|            3636|
|         79|            9318|
|         80|           17667|
+-----------+----------------+



In [47]:
#Sauvegarder les données récupérées dans un csv
nbVaccByAge.toPandas().to_csv("./data/save/nb_vacc_by_age.csv", header=True, index=False)

In [48]:
#lire le csv sauvegardé

#nbVaccByAge_data = spark.read.load("./data/save/nb_vacc_by_age.csv", format="csv", header=True, inferSchema="True")

nbVaccByAge_data = pd.read_csv('./data/save/nb_vacc_by_age.csv')

In [49]:
type(nbVaccByAge_data)

pandas.core.frame.DataFrame

In [50]:
nbVaccByAge_data.head()

,clage_vacsi,max(n_cum_dose1)
0,24,325
1,29,809
2,39,2218
3,49,3169
4,59,9925


In [51]:
st.title("**♟**Explore E-com dashboard**♟**")
st.write("ABCD LE PROJET PYTHON")